In [3]:
import ipynb.fs.full.preprocess_eda as preprocess

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The length of the training data:  7500
The length of the testing data:  4500
The length of the validation data:  3000


In [4]:
from deep_translator import GoogleTranslator

In [5]:
import os

In [6]:
PATH = r"D:\MScDataScience\7.Data_Science_Project\SourceCode\augment_data\Back_Translation"
traget_languages = ["hi", "fr", "de", "es"]

In [7]:
class back_translate():
    def __init__(self, traget_languages, path, data):
        self.traget_languages = traget_languages
        self.path = path
        self.data = data

    def google_translate(self, original_text, traget):
        translated_text = GoogleTranslator(source="en", target=traget).translate_batch(original_text)
        return GoogleTranslator(source=traget, target="en").translate_batch(translated_text)
    
    def augment(self):
        original_text = self.data.iloc[:, 0].to_list()
        labels = self.data.iloc[:, 1]
        for traget in self.traget_languages:
            print(f"[INFO] Augmenting data using language {traget} ...")
            preprocess.pd.DataFrame({
                "Query": self.google_translate(original_text, traget),
                "Intent": labels
            }).to_csv(os.path.join(self.path, 
                                   f"back_translation_augment_data_{traget}.csv"))
            print("\t Augmented and saved to the disk.")

In [6]:
translate_augment = back_translate(traget_languages, PATH, preprocess.train_df)

In [ ]:
translate_augment.augment()

[INFO] Augmenting data using language hi ...
	 Augmented and saved to the disk.
[INFO] Augmenting data using language fr ...
	 Augmented and saved to the disk.
[INFO] Augmenting data using language de ...
	 Augmented and saved to the disk.
[INFO] Augmenting data using language es ...


In [12]:
translate_augment_obj = back_translate(["es"], PATH, preprocess.train_df)

In [13]:
translate_augment_obj.augment()

[INFO] Augmenting data using language es ...
	 Augmented and saved to the disk.


In [8]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.layers import Flatten

In [9]:
augment_df = preprocess.pd.read_csv(os.path.join(PATH, 
                                                 "back_translation_augment_data_hi.csv"), 
                                                 index_col=0)

In [10]:
train_df = preprocess.pd.concat([preprocess.train_df, augment_df], ignore_index=True)

In [11]:
train_df

,Query,Intent
0,can you walk me through setting up direct depo...,direct_deposit
1,i want to switch to direct deposit,direct_deposit
2,set up direct deposit for me,direct_deposit
3,how do i go about setting up direct deposit,direct_deposit
4,i need to get my paycheck direct deposited to ...,direct_deposit
...,...,...
14995,I want to change my credit limit,credit_limit_change
14996,I want to increase my Bank of America credit l...,credit_limit_change
14997,Can I increase the credit limit on my Old Navy...,credit_limit_change
14998,Can I increase the credit limit on my Kohl's c...,credit_limit_change


In [12]:
train_obj = preprocess.pre_process(train_df)
val_obj = preprocess.pre_process(preprocess.val_df)
test_obj = preprocess.pre_process(preprocess.test_df)

x_train = train_obj.preprocess()
x_train = train_obj.lemmatise()

x_val = val_obj.preprocess()
x_val = val_obj.lemmatise()

x_test = test_obj.preprocess()
x_test = test_obj.lemmatise()

y_train = train_obj.encode_class()
y_val = val_obj.encode_class()
y_test = test_obj.encode_class()

In [13]:
vocab_size = 20000
sequence_length = 30

In [14]:
textvector_layer = TextVectorization(max_tokens=vocab_size, 
                                     ngrams=(1, 2, 3),
                                     output_mode="int", 
                                    #  pad_to_max_tokens=True)
                                    output_sequence_length=sequence_length)

In [15]:
textvector_layer.adapt(preprocess.pd.concat([x_train, x_val, x_test],
                                            ignore_index=True))

In [23]:
model = Sequential()
model.add(textvector_layer)
model.add(Embedding(vocab_size+2, 128))
# model.add(Conv1D(64, 
#                   kernel_size=3, 
#                   activation="relu", 
#                   padding="same"))
# model.add(Dropout(0.2))
# model.add(Conv1D(32, 
#                   kernel_size=2, 
#                   activation="relu", 
#                   padding="same"))
# model.add(Dropout(0.2))
# model.add(LSTM(100, 
#                 recurrent_dropout=0.2,
#                 activation="tanh", 
#                 return_sequences=True))
model.add(LSTM(100,
               dropout=0.2,
               recurrent_dropout=0.2,
               activation="tanh"))
# model2.add(LSTM(100, activation="tanh"))
# model2.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(150, activation="softmax"))

model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.001), 
                      loss=keras.losses.CategoricalCrossentropy(),
                      metrics=["accuracy"])


epochs = 150
batch_size = 64

history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=epochs, 
                    batch_size=batch_size,
                    callbacks=[EarlyStopping(monitor='val_accuracy', 
                                             patience=15, 
                                             min_delta=0.0001)],
                    shuffle=True)

Epoch 1/150
235/235 ━━━━━━━━━━━━━━━━━━━━ 16s 51ms/step - accuracy: 0.0079 - loss: 4.9748 - val_accuracy: 0.0203 - val_loss: 4.5616
Epoch 2/150
235/235 ━━━━━━━━━━━━━━━━━━━━ 13s 53ms/step - accuracy: 0.0212 - loss: 4.4162 - val_accuracy: 0.0277 - val_loss: 4.4186
Epoch 3/150
235/235 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.0313 - loss: 4.1324 - val_accuracy: 0.0473 - val_loss: 3.9112
Epoch 4/150
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.0505 - loss: 3.6958 - val_accuracy: 0.0723 - val_loss: 3.7165
Epoch 5/150
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.0720 - loss: 3.4063 - val_accuracy: 0.0887 - val_loss: 3.5035
Epoch 6/150
235/235 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.1057 - loss: 3.1427 - val_accuracy: 0.1243 - val_loss: 3.3182
Epoch 7/150
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.1434 - loss: 2.9150 - val_accuracy: 0.1820 - val_loss: 3.0856
Epoch 8/150
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.2291 - loss: 2.

In [84]:
textvector_layer1 = TextVectorization(max_tokens=vocab_size, 
                                     ngrams=(1, 2, 3),
                                     output_mode="tf_idf", 
                                     pad_to_max_tokens=True)
                                    # output_sequence_length=sequence_length)

In [85]:
textvector_layer1.adapt(preprocess.pd.concat([x_train, x_val, x_test],
                                            ignore_index=True))

In [ ]:
model2 = Sequential()
model2.add(textvector_layer1)
model2.add(Embedding(vocab_size+2, 128))
# model.add(Conv1D(64, 
#                   kernel_size=3, 
#                   activation="relu", 
#                   padding="same"))
# model.add(Dropout(0.2))
# model.add(Conv1D(32, 
#                   kernel_size=2, 
#                   activation="relu", 
#                   padding="same"))
# model.add(Dropout(0.2))
# model.add(LSTM(100, 
#                 recurrent_dropout=0.2,
#                 activation="tanh", 
#                 return_sequences=True))
model2.add(LSTM(100,
               dropout=0.2,
               recurrent_dropout=0.2,
               activation="tanh"))
# model2.add(LSTM(100, activation="tanh"))
# model2.add(Dropout(0.2))
model2.add(Flatten())
model2.add(Dense(150, activation="softmax"))

model2.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.001), 
                      loss=keras.losses.CategoricalCrossentropy(),
                      metrics=["accuracy"])


epochs = 150
batch_size = 64

history = model2.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=epochs, 
                    batch_size=batch_size,
                    callbacks=[EarlyStopping(monitor='val_accuracy', 
                                             patience=15, 
                                             min_delta=0.0001)],
                    shuffle=True)

Epoch 1/150
  1/235 ━━━━━━━━━━━━━━━━━━━━ 102:15:54 1573s/step - accuracy: 0.0156 - loss: 5.0113

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
tfidf_vectorizer = TfidfVectorizer(input="content", stop_words="english", ngram_range=(1, 3))

In [ ]:
import itertools

In [50]:
list(itertools.chain.from_iterable(x_train.str.split().to_list()))

['walk',
 'setting',
 'direct',
 'deposit',
 'bank',
 'internet',
 'saving',
 'account',
 'want',
 'switch',
 'direct',
 'deposit',
 'set',
 'direct',
 'deposit',
 'go',
 'setting',
 'direct',
 'deposit',
 'need',
 'get',
 'paycheck',
 'direct',
 'deposited',
 'chase',
 'account',
 'step',
 'set',
 'direct',
 'deposit',
 'chase',
 'account',
 'would',
 'like',
 'set',
 'direct',
 'deposit',
 'direct',
 'deposit',
 'check',
 'need',
 'set',
 'direct',
 'deposit',
 'would',
 'like',
 'paycheck',
 'direct',
 'deposited',
 'chase',
 'account',
 'direct',
 'deposit',
 'set',
 'teach',
 'set',
 'direct',
 'deposit',
 'show',
 'turn',
 'direct',
 'deposit',
 'direct',
 'deposit',
 'information',
 'want',
 'set',
 'direct',
 'deposit',
 'first',
 'hawaiian',
 'bank',
 'account',
 'tell',
 'set',
 'direct',
 'deposit',
 'would',
 'like',
 'know',
 'setting',
 'direct',
 'deposit',
 'tell',
 'set',
 'direct',
 'deposit',
 'tell',
 'set',
 'direct',
 'deposit',
 'paycheck',
 'help',
 'setting',
 

In [51]:
tfidf_vectorizer.fit(list(itertools.chain.from_iterable(x_train.str.split().to_list())))

TfidfVectorizer(ngram_range=(1, 3), stop_words='english')

In [ ]:
x = tfidf_vectorizer.transform(x_train[4].split()).toarray()

In [71]:
len(x_train[4].split())

7

In [68]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [67]:
x.shape

(7, 3396)

In [75]:
tfidf_vectorizer.vocabulary_

{'walk': 3276,
 'setting': 2702,
 'direct': 876,
 'deposit': 829,
 'bank': 257,
 'internet': 1515,
 'saving': 2647,
 'account': 24,
 'want': 3283,
 'switch': 2975,
 'set': 2700,
 'need': 2021,
 'paycheck': 2205,
 'deposited': 830,
 'chase': 517,
 'step': 2895,
 'like': 1709,
 'check': 525,
 'teach': 3006,
 'turn': 3158,
 'information': 1485,
 'hawaiian': 1344,
 'tell': 3014,
 'know': 1634,
 'help': 1360,
 'directly': 878,
 'make': 1814,
 'needed': 2022,
 'guidance': 1305,
 'come': 630,
 'instant': 1500,
 'really': 2433,
 'arrange': 183,
 'specific': 2838,
 'way': 3295,
 'money': 1953,
 'market': 1842,
 'pay': 2204,
 'depost': 831,
 'job': 1577,
 'great': 1285,
 'western': 3313,
 'websearch': 3298,
 'allowed': 111,
 'carry': 479,
 'american': 127,
 'airline': 88,
 'rule': 2601,
 'united': 3192,
 'carryon': 480,
 'delta': 817,
 'uniteds': 3193,
 'policy': 2289,
 'limit': 1713,
 'fly': 1153,
 'internationally': 1514,
 'brittish': 378,
 'airwaiys': 90,
 'limitation': 1714,
 'luggage': 1785

In [72]:
tfidf_vectorizer.get_feature_names_out()

array(['aa', 'aaa', 'abandon', ..., 'ziti', 'zone', 'zoo'], dtype=object)

In [74]:
preprocess.pd.DataFrame(x, columns=tfidf_vectorizer.get_feature_names_out())

,aa,aaa,abandon,abc,able,aborgate,abort,abortion,abroad,abrogate,...,zepher,zephers,zephyr,zero,zesty,zeus,zippy,ziti,zone,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
tfidf_vectorizer.inverse_transform(x)

[array(['need'], dtype='<U19'),
 array([], dtype='<U19'),
 array(['paycheck'], dtype='<U19'),
 array(['direct'], dtype='<U19'),
 array(['deposited'], dtype='<U19'),
 array(['chase'], dtype='<U19'),
 array(['account'], dtype='<U19')]

In [77]:
x_train[4].split()

['need', 'get', 'paycheck', 'direct', 'deposited', 'chase', 'account']

In [81]:
tfidf_vectorizer.transform(x_train).toarray().shape

(15000, 29205)

In [95]:
textvector_layer1(x_test[7]).numpy()

array([50.87265,  0.     ,  0.     , ...,  0.     ,  0.     ,  0.     ],
      dtype=float32)

In [97]:
len(textvector_layer1.get_vocabulary())

20000